In [ ]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


change into scikit 1.0.2

In [ ]:
!pip install scikit-learn==1.0.2

You should consider upgrading via the '/home/desenfirman/miniconda3/bin/python3.8 -m pip install --upgrade pip' command.


In [ ]:
import pickle
import pandas as pd

In [ ]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/home/desenfirman/miniconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/desenfirman/miniconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [80]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

## Question 1

In [81]:
print(y_pred.mean())

16.191691679979066


## Question 2

In [82]:
year = 2022
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration,ride_id
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173,82,None,B00021,10.666667,2022/02_1
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173,56,None,B00021,14.566667,2022/02_2
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82,129,None,B00021,7.950000,2022/02_3
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1,225,None,B00037,13.800000,2022/02_4
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1,61,None,B00037,8.966667,2022/02_5
...,...,...,...,...,...,...,...,...,...
1037687,B03282,2021-02-28 23:01:16,2021-02-28 23:14:48,-1,31,None,B01717,13.533333,2022/02_1037687
1037688,B03282,2021-02-28 23:36:10,2021-02-28 23:47:38,-1,169,None,B01717,11.466667,2022/02_1037688
1037689,B03285,2021-02-28 23:18:36,2021-02-28 23:43:59,28,171,None,B03285,25.383333,2022/02_1037689
1037690,B03285,2021-02-28 23:26:34,2021-02-28 23:44:37,16,252,None,B03285,18.050000,2022/02_1037690


In [83]:
df_result = df[['ride_id']].copy()
df_result['prediction'] = y_pred
df_result

,ride_id,prediction
1,2022/02_1,14.539865
2,2022/02_2,13.740422
3,2022/02_3,15.593339
4,2022/02_4,15.188118
5,2022/02_5,13.817206
...,...,...
1037687,2022/02_1037687,12.433246
1037688,2022/02_1037688,12.910885
1037689,2022/02_1037689,15.834923
1037690,2022/02_1037690,16.783176


In [84]:
df_result.to_parquet(
    'fhv_tripdata_2022-02_ride_id_predictions.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [85]:
!ls -lh

total 19M
-rw-rw-r-- 1 desenfirman desenfirman 241 Jun 25 21:14 Dockerfile
-rw-rw-r-- 1 desenfirman desenfirman 19M Jun 25 21:25 fhv_tripdata_2022-02_ride_id_predictions.parquet
-rw-rw-r-- 1 desenfirman desenfirman 18K Jun 25 18:34 model.bin
-rw-rw-r-- 1 desenfirman desenfirman 188 Jun 25 21:24 Pipfile
-rw-r--r-- 1 desenfirman desenfirman 18K Jun 25 21:24 Pipfile.lock
-rw-rw-r-- 1 desenfirman desenfirman 35K Jun 25 21:13 starter.ipynb
-rw-rw-r-- 1 desenfirman desenfirman 929 Jun 25 21:05 starter.py


## Question 3

In [86]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 2161 bytes to starter.py


## Question 4

In [87]:
!pip install pipenv

You should consider upgrading via the '/home/desenfirman/miniconda3/bin/python3.8 -m pip install --upgrade pip' command.


In [97]:
!echo Y | pipenv install scikit-learn==1.0.2 flask --python=3.9 

Installing scikit-learn==1.0.2...
Adding scikit-learn to Pipfile's [packages]...
✔ Installation Succeeded 
Installing flask...
Adding flask to Pipfile's [packages]...
✔ Installation Succeeded 
Installing dependencies from Pipfile.lock (e37673)...
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 0/0 — 00:00:00
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [ ]:
!cat Pipfile.lock | tail -n +140 | head -n 25

        },
        "scikit-learn": {
            "hashes": [
                "sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b",
                "sha256:158faf30684c92a78e12da19c73feff9641a928a8024b4fa5ec11d583f3d8a87",
                "sha256:16455ace947d8d9e5391435c2977178d0ff03a261571e67f627c8fee0f9d431a",
                "sha256:245c9b5a67445f6f044411e16a93a554edc1efdcce94d3fc0bc6a4b9ac30b752",
                "sha256:285db0352e635b9e3392b0b426bc48c3b485512d3b4ac3c7a44ec2a2ba061e66",
                "sha256:2f3b453e0b149898577e301d27e098dfe1a36943f7bb0ad704d1e548efc3b448",
                "sha256:46f431ec59dead665e1370314dbebc99ead05e1c0a9df42f22d6a0e00044820f",
                "sha256:55f2f3a8414e14fbee03782f9fe16cca0f141d639d2b1c1a36779fa069e1db57",
                "sha256:5cb33fe1dc6f73dc19e67b264dbb5dde2a0539b986435fdd78ed978c14654830",
                "sha256:75307d9ea39236cad7eea87143155eea24d48f93f3a2f9389c817f7019f00705",
                "sha256:7626a

## Question 5

In [93]:
!cat starter.py

#!/usr/bin/env python
# coding: utf-8
import pickle
import sys
import pandas as pd


categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

def run():
    year = int(sys.argv[1]) # 2021
    month = int(sys.argv[2]) # 3
    with open('model.bin', 'rb') as f_in:
        dv, lr = pickle.load(f_in)


    df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')


    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = lr.predict(X_val)


    print(y_pred.mean())


if __name__ == '__main__':
    run()

In [94]:
!python starter.py 2021 3

16.298821614015107


## Question 6

In [98]:
!pipenv install pandas pyarrow

Installing pandas...
Adding pandas to Pipfile's [packages]...
✔ Installation Succeeded 
Installing pyarrow...
⠧ Installing pyarrow...

In [89]:
!cat Pipfile

[[source]]
url = "https://pypi.org/simple"
verify_ssl = true
name = "pypi"

[packages]
scikit-learn = "==1.0.2"
flask = "*"
pandas = "*"

[dev-packages]

[requires]
python_version = "3.9"


In [90]:
!cat Dockerfile

FROM agrigorev/zoomcamp-model:mlops-3.9.7-slim

RUN pip install -U pip
RUN pip install pipenv 

COPY [ "Pipfile", "Pipfile.lock", "./" ]

RUN pipenv install --system --deploy

COPY starter.py starter.py

ENTRYPOINT [ "python", "starter.py" ]

In [95]:
!docker build -t mlopszoomcamp-test-starter .

Sending build context to Docker daemon  19.79MB
Step 1/7 : FROM agrigorev/zoomcamp-model:mlops-3.9.7-slim
 ---> 8cffad87c549
Step 2/7 : RUN pip install -U pip
 ---> Using cache
 ---> 1f991134873e
Step 3/7 : RUN pip install pipenv
 ---> Using cache
 ---> 5ba9bad5e3bc
Step 4/7 : COPY [ "Pipfile", "Pipfile.lock", "./" ]
 ---> Using cache
 ---> 725ff7c5afab
Step 5/7 : RUN pipenv install --system --deploy
 ---> Using cache
 ---> baca3aac4015
Step 6/7 : COPY starter.py starter.py
 ---> 9a043f2c6931
Step 7/7 : ENTRYPOINT [ "python", "starter.py" ]
 ---> Running in c05b494dd236
Removing intermediate container c05b494dd236
 ---> 4db96f537f1c
Successfully built 4db96f537f1c
Successfully tagged mlopszoomcamp-test-starter:latest


In [96]:
!docker run -t mlopszoomcamp-test-starter 2021 4

Traceback (most recent call last):
  File "/app/starter.py", line 41, in <module>
    run()
  File "/app/starter.py", line 29, in run
    df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')
  File "/app/starter.py", line 11, in read_data
    df = pd.read_parquet(filename)
  File "/usr/local/lib/python3.9/site-packages/pandas/io/parquet.py", line 491, in read_parquet
    impl = get_engine(engine)
  File "/usr/local/lib/python3.9/site-packages/pandas/io/parquet.py", line 52, in get_engine
    raise ImportError(
ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use p